This notebook will try to reach better accuracy of the image calssification of the dataset by utilizing pretrained image recognition model. 

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import scipy as sp
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")



Python Platform: macOS-13.2.1-arm64-arm-64bit
Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.9.6 (default, Oct 18 2022, 12:41:40) 
[Clang 14.0.0 (clang-1400.0.29.202)]
Pandas 2.0.0
Scikit-Learn 1.2.2
SciPy 1.10.1
GPU is available


In [2]:
print("Num GPUs Available: ", len(tf.test.gpu_device_name()))

Metal device set to: Apple M1 Max
Num GPUs Available:  13


In [7]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Some hyperparameters set up:

IMG_SIZE=256
# Set up the ImageNet pre-trained model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the layers in the pre-trained model
for layer in resnet.layers:
    layer.trainable = False

# Build a new model on top of the pre-trained model
model = Sequential()
model.add(resnet)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=1e-2), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 131072)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               16777344  
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 5)                 645       
                                                                 
Total params: 40,365,701
Trainable params: 16,777,989
Non-trainable params: 23,587,712
_________________________________________________________________


In [5]:
batch_size=32
DATA_DIR='../dataset/inputs_resized/flowers'
# Create data generators for data augmentation and validation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rescale=1./255,       
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,  # rescale it (I guess for the better numbers for the gradients (no blow ups))
        validation_split=0.2,  # split the dataset into train and validation parts
        vertical_flip=False)  # randomly flip images

# datagen.fit(x_train)

# Load training and validation data using the data generator
train_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # specify the subset as 'training' for the training data

val_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # specify the subset as 'validation' for the validation data

# # Set up the data generators
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                    shear_range=0.2,
#                                    zoom_range=0.2,
#                                    horizontal_flip=True)
# test_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow_from_directory('path/to/train/directory',
#                                                     target_size=(224, 224),
#                                                     batch_size=32,
#                                                     class_mode='categorical')

# test_generator = test_datagen.flow_from_directory('path/to/test/directory',
#                                                   target_size=(224, 224),
#                                                   batch_size=32,
#                                                   class_mode='categorical')


Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [10]:


# Train the model

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
batch_size=32
epochs=50
History = model.fit(train_data, batch_size=batch_size,
                     epochs=epochs, validation_data=val_data, verbose=1,
                     callbacks=[reduce_lr])

# history = model.fit_generator(train_data,
#                               steps_per_epoch=100,
#                               epochs=40,
#                               validation_data=val_data,
#                               validation_steps=50,
#                               verbose=1)



Epoch 1/50
109/109 [==============================] - 30s 275ms/step - loss: 1.5987 - accuracy: 0.2436 - val_loss: 1.5984 - val_accuracy: 0.2442 - lr: 0.0010
Epoch 2/50
109/109 [==============================] - 30s 275ms/step - loss: 1.5987 - accuracy: 0.2436 - val_loss: 1.5983 - val_accuracy: 0.2442 - lr: 0.0010
Epoch 3/50
109/109 [==============================] - 31s 281ms/step - loss: 1.5986 - accuracy: 0.2436 - val_loss: 1.5983 - val_accuracy: 0.2442 - lr: 0.0010
Epoch 4/50
109/109 [==============================] - 30s 282ms/step - loss: 1.5986 - accuracy: 0.2436 - val_loss: 1.5983 - val_accuracy: 0.2442 - lr: 0.0010
Epoch 5/50
109/109 [==============================] - 30s 276ms/step - loss: 1.5986 - accuracy: 0.2436 - val_loss: 1.5983 - val_accuracy: 0.2442 - lr: 5.0000e-04
Epoch 6/50
109/109 [==============================] - 30s 275ms/step - loss: 1.5985 - accuracy: 0.2436 - val_loss: 1.5983 - val_accuracy: 0.2442 - lr: 5.0000e-04
Epoch 7/50
109/109 [========================

KeyboardInterrupt: 

In [ ]:

# Evaluate the model
score = model.evaluate_generator(test_generator, steps=50)
print('Test accuracy:', score[1])
